In [1]:
from syft import Worker
import syft as sy
import numpy as np
import pandas as pd
worker = Worker.named("test-domain-1", processes=1, reset=False)
root_domain_client = worker.root_client

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/PySyft


SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.Data

# Summary

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at /var/log/popularity-contest.

## Get mocks

In [2]:
guest_domain_client = worker.guest_client
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

In [3]:
ds = guest_domain_client.datasets[0]

In [4]:
asset = ds.assets[0]

In [5]:
popcon = asset.mock.syft_action_data

## Parsing Unix timestamps

The colums are the access time, created time, package name, recently used program, and a tag



In [6]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,135372952,135215089,ubuntu-extras-keyring,/usr/bin/page,<OLD>
1,135496877,135919911,libbsd0,/lib/init/upstart-job,<OLD>
2,135785604,135211551,ubuntu-extras-keyring,/lib/init/upstart-job,<OLD>
3,135184838,135209012,libghc-stm-dev,/usr/bin/onboard,<OLD>
4,135439842,135625774,geoclue-ubuntu-geoip,/usr/bin/page,<OLD>


The magical part about parsing timestamps in pandas is that numpy datetimes are already stored as Unix timestamps. So all we need to do is tell pandas that these integers are actually datetimes -- it doesn't need to do any conversion at all.

We need to convert these to ints to start:

In [7]:
popcon['atime'] = popcon['atime'].astype(int)
popcon['ctime'] = popcon['ctime'].astype(int)

Every numpy array and pandas series has a dtype -- this is usually `int64`, `float64`, or `object`. Some of the time types available are `datetime64[s]`, `datetime64[ms]`, and `datetime64[us]`. There are also `timedelta` types, similarly.

We can use the `pd.to_datetime` function to convert our integer timestamps into datetimes. This is a constant-time operation -- we're not actually changing any of the data, just how pandas thinks about it.

In [8]:
popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')

If we look at the dtype now, it's `<M8[ns]`. As far as I can tell `M8` is secret code for `datetime64`.

In [9]:
popcon['atime'].dtype

dtype('<M8[ns]')

So now we can look at our `atime` and `ctime` as dates!

In [10]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,1974-04-16 19:35:52,1974-04-14 23:44:49,ubuntu-extras-keyring,/usr/bin/page,<OLD>
1,1974-04-18 06:01:17,1974-04-23 03:31:51,libbsd0,/lib/init/upstart-job,<OLD>
2,1974-04-21 14:13:24,1974-04-14 22:45:51,ubuntu-extras-keyring,/lib/init/upstart-job,<OLD>
3,1974-04-14 15:20:38,1974-04-14 22:03:32,libghc-stm-dev,/usr/bin/onboard,<OLD>
4,1974-04-17 14:10:42,1974-04-19 17:49:34,geoclue-ubuntu-geoip,/usr/bin/page,<OLD>


Now suppose we want to look at all packages that aren't libraries.

First, I want to get rid of everything with timestamp 0. Notice how we can just use a string in this comparison, even though it's actually a timestamp on the inside? That is because pandas is amazing.


In [11]:
popcon = popcon[popcon['atime'] > '1970-01-01']

Now we can use pandas' magical string abilities to just look at rows where the package name doesn't contain 'lib'.

In [12]:
nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]

In [13]:
nonlibraries.sort_values('ctime', ascending=False)[:10]

,atime,ctime,package-name,mru-program,tag
595,1974-04-12 15:34:05,1974-04-24 01:44:39,geoclue-ubuntu-geoip,/lib/init/upstart-job,nan
1723,1974-04-20 07:54:39,1974-04-24 01:40:37,ubuntu-extras-keyring,/lib/init/upstart-job,nan
341,1974-04-18 09:54:36,1974-04-24 01:10:03,geoclue-ubuntu-geoip,/usr/bin/page,<OLD>
2378,1974-04-20 18:38:20,1974-04-24 00:24:57,ubuntu-extras-keyring,/usr/bin/opam,nan
1514,1974-04-20 23:42:15,1974-04-23 23:47:59,geoclue-ubuntu-geoip,/usr/bin/onboard,nan
1933,1974-04-13 16:03:19,1974-04-23 23:42:52,geoclue-ubuntu-geoip,/usr/bin/opam,nan
2251,1974-04-13 04:23:23,1974-04-23 22:58:21,ubuntu-extras-keyring,/lib/init/upstart-job,<OLD>
843,1974-04-12 12:00:05,1974-04-23 22:38:54,ubuntu-extras-keyring,/usr/bin/onboard,nan
1790,1974-04-13 19:34:51,1974-04-23 22:33:06,ubuntu-extras-keyring,/lib/init/upstart-job,<RECENT-CTIME>
44,1974-04-21 00:03:37,1974-04-23 22:20:54,ubuntu-extras-keyring,/lib/init/upstart-job,<RECENT-CTIME>


Okay, cool, it says that I I installed ddd recently. And postgresql! I remember installing those things. Neat.

The whole message here is that if you have a timestamp in seconds or milliseconds or nanoseconds, then you can just "cast" it to a `'datetime64[the-right-thing]'` and pandas/numpy will take care of the rest.

## Putting it together

Now we want to request the full code execution.

Let's put all that together, to prove how easy it is. 6 lines of magical pandas!

In [14]:
@sy.syft_function(input_policy=sy.ExactMatch(popcon=ds.assets[0]),
                  output_policy=sy.SingleExecutionExactOutput())
def find_recently_installed(popcon):
    import pandas as pd
    popcon['atime'] = popcon['atime'].astype(int)
    popcon['ctime'] = popcon['ctime'].astype(int)
    popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
    popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')
    popcon = popcon[popcon['atime'] > '1970-01-01']
    nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]
    return nonlibraries.sort_values('ctime', ascending=False)[:10]

Request code execution

In [15]:
req = guest_domain_client.api.services.code.request_code_execution(find_recently_installed)

In [16]:
submitted_code = guest_domain_client.code[0]

In [17]:
assert guest_domain_client.api.services.code.get_all()

Create and submit project

In [18]:
new_project = sy.Project(name="Pandas Chapter 8",
                         description="Hi, I would like to get some insights about the installed programs")

In [19]:
new_project.add_request(obj=submitted_code, permission=sy.UserCodeStatus.EXECUTE)

In [20]:
guest_domain_client.submit_project(new_project)

<class 'syft.core.node.new.response.SyftSuccess'>: Project Submitted